In [ ]:
library(tidyr)
library(readr)
library(plyr)
library(dplyr)
#library(stringr)
library(rgdal)
library(ggplot2)
library(sf)

In [ ]:
daily<-read_csv("https://raw.githubusercontent.com/ericnost/gfw/setup/data/2016-12-22.csv")

In [ ]:
countries <- read_sf("https://raw.githubusercontent.com/ericnost/gfw/setup/data/countries.geojson")
wcpfc_map <- read_sf("https://raw.githubusercontent.com/ericnost/gfw/setup/data/wcpfc.geojson") 
spc <- read_sf("https://raw.githubusercontent.com/ericnost/gfw/setup/data/spc.geojson")

In [ ]:
map = ggplot() + 
geom_sf(data = wcpfc_map, aes(fill = blue)) +
geom_sf(data = spc, aes(fill = green)) +
geom_sf(data = countries, aes(fill = NAME)) +
theme(legend.position = "none") 
map

In [ ]:
# Load shapefiles
countries <- read_sf("https://raw.githubusercontent.com/ericnost/gfw/setup/data/countries.geojson")#readOGR(dsn="https://raw.githubusercontent.com/ericnost/gfw/setup/data/countries.geojson")
plot(countries, border="wheat3", col="wheat1")

wcpfc_map <- readOGR(dsn="https://raw.githubusercontent.com/ericnost/gfw/setup/data/wcpfc.geojson") #<- readOGR(dsn = "https://github.com/ericnost/gfw/tree/setup/shapefiles/Pacific", layer = "WCPFC") 
plot(wcpfc_map, border="wheat3", col="wheat2", add=T)

spc <- readOGR(dsn="https://raw.githubusercontent.com/ericnost/gfw/setup/data/spc.geojson")
plot(spc, border="wheat3", col="wheat3", add=T)

In [ ]:
# Load GFW's main vessel database described here: https://globalfishingwatch.org/datasets-and-code/vessel-identity/
vessels_full <- read_csv("https://raw.githubusercontent.com/ericnost/gfw/setup/data/vessels.csv")

# Load the WCPFC database of Registered Fishing Vessels
wcpfc <- read_csv("https://raw.githubusercontent.com/ericnost/gfw/setup/data/RFV_database_standardized.csv")

In [ ]:
head(wcpfc)

In [ ]:
# Zoom in to a selected company. For now, you have to type out the name of the company you want to focus on...
# exactly as it is spelled in the database. The default option is: Dongwon.
# 
#ky<-wcpfc[grep("Kyokuyo", wcpfc$`Owner Name`),] # 5 vessels
dongwon<-wcpfc[grep("Dongwon", wcpfc$`Owner Name`),] # 44 vessels
dongwon
# Merge dongwon and ky to look at both of these 'keystone actors' simultaneously
#dongwon<-rbind(dongwon,ky)

In [ ]:
#Get the identifying info for Dongwon vessels, as described in the WCPFC RFV.
dongwon_imos<-dongwon$`IMO-LR`
dongwon_ircs<-dongwon$IRCS

filtered<-vessels_full[which(vessels_full$imo %in% dongwon_imos | vessels_full$callsign %in% dongwon_ircs ),]

dailies_filtered<-daily[which(daily$mmsi %in% filtered$mmsi),] #filtered to dongwon
dailies_extracted<-daily[-which(daily$mmsi %in% filtered$mmsi),] #all other vessels

In [ ]:
#join vessels and effort
dongwon_fishing<-merge(dailies_filtered, filtered, by = "mmsi")
allother_fishing<-merge(dailies_extracted, vessels_full, by ="mmsi")
dongwon_fishing

In [ ]:
#aggregate by lat/long and vessel - all others
aggregated<-allother_fishing %>%
  group_by(lat_bin, lon_bin) %>% #, flag #aggregated at specific lat long bins
  summarise(totalFishingHours=sum(fishing_hours), flags=n_distinct(flag), count=n_distinct(mmsi)) 
#aggregated = total fishing effort at this lat long, number of unique flags and mmsis at this lat long
aggregated$lat_bin<-(aggregated$lat_bin/10)+.05
aggregated$lon_bin<-(aggregated$lon_bin/10)+.05

#max - produce a set of points with the max ALL OTHER FISHING for each lat/lon bin and report the flag
aggregated_max<-allother_fishing %>%
  group_by(lat_bin, lon_bin, flag) %>% #aggregated at specific lat long bins and flags
  summarise(totalFishingHours=sum(fishing_hours)) 
aggregated_max<-aggregated_max %>% 
  group_by(lat_bin, lon_bin) %>%
  filter(totalFishingHours == max(totalFishingHours))  #max here 553 -365
#total fishing effort at this lat long and the country that did it
aggregated_max$lat_bin<-(aggregated_max$lat_bin/10)+.05
aggregated_max$lon_bin<-(aggregated_max$lon_bin/10)+.05

aggregated
aggregated_max

In [ ]:
#aggregate by lat/long and vessel - - Dongwon
dongwon_aggregated<-dongwon_fishing %>%
  group_by(lat_bin, lon_bin) %>% #aggregated at specific lat long bins
  summarise(totalFishingHours=sum(fishing_hours), flag="DONG", count=n_distinct(mmsi)) 
#total fishing effort at this lat long, number of unique Dongwon mmsis at this lat long
dongwon_aggregated$lat_bin<-(dongwon_aggregated$lat_bin/10)+.05
dongwon_aggregated$lon_bin<-(dongwon_aggregated$lon_bin/10)+.05
dongwon_aggregated

In [ ]:
#dongwon and all other together maxed.... 
#This will help us show the biggest flag or company (Dongwon) for each lat/long bin
dongwon_aggregated<-dongwon_aggregated[,-c(5)]
colnames(dongwon_aggregated)<-c("lat_bin", "lon_bin", "totalFishingHours", "flag")
total<-rbind(aggregated_max, dongwon_aggregated)
total<-total %>% 
  group_by(lat_bin, lon_bin) %>%
  filter(totalFishingHours == max(totalFishingHours)) 
total

In [ ]:
# Output
write_csv(aggregated, "all_other_fishing_total.csv")
write_csv(aggregated_max, "all_other_fishing_max.csv")
#write_csv(aggflag, "allotherfishing-specificflags-TOTAL.csv")
write_csv(dongwon_aggregated, "dongwon.csv")
write_csv(total, "all_fishing_max.csv")

In [ ]:
total <- mutate(total, flag=recode(flag, 
                         "DONG"="DONG",
                         "CHN"="CHINA",
                         .default="OTHR"))
total

In [ ]:
# create points out of total data
total_pts = st_as_sf(total, coords = c("lon_bin", "lat_bin"), crs = 4326)

In [ ]:
#standardize the CRS
#st_transform(total, crs = 4326)
wcpfc_map<-st_transform(wcpfc_map, crs = 4326)
spc<-st_transform(spc, crs = 4326)
countries<-st_transform(countries, crs = 4326)

In [ ]:
#clip
clipped_total<-st_intersection(total_pts, st_buffer(spc, 0))
clipped_total

In [ ]:
#projection
p = CRS("+proj=aea +lat_0=-30 +lat_1=30 +lon_0=-180")
wcpfc_map<-st_transform(wcpfc_map, p)
spc<-st_transform(spc, p)
countries<-st_transform(countries, p)
clipped_total<-st_transform(clipped_total, p)

In [ ]:
#DEVELOP COLOR SCALE, CHANGE FROM + to POINTS
#library(RColorBrewer) # creates nice color schemes
#library(classInt) 
#nclr <- 4
#plotclr <- brewer.pal(nclr,"BuPu")
#class <- classIntervals(clipped_total$totalFishingHours, nclr, style="quantile") #clipped #dongwon_clipped
#colcode <- findColours(class, plotclr)
#Create a custom color scale
library(RColorBrewer)
myColors <- brewer.pal(3,"Set1")
names(myColors) <- levels(clipped_total$flag)
colScale <- scale_colour_manual(name = "grp",values = myColors)



#plot(countries, border="wheat3", col="wheat1")
#plot(wcpfc_map, border="wheat3", col="lightblue")
#plot(spc, border="wheat3", col="red", add=T)
#plot(clipped_total["flag"], pal = sf.colors(3, categorical = TRUE), 
#     pch=16, key.pos = 4, key.width = lcm(4.5), axes = TRUE, graticule = TRUE)
#plot
map = ggplot() + 
geom_sf(data = wcpfc_map, fill = "blue", alpha=.1) +
geom_sf(data = spc, fill = "green", alpha=.1) +
#geom_sf(data = countries, aes(fill = NAME)) +
geom_point(
    data=clipped_total,
    aes(color = flag, geometry = geometry),
    stat = "sf_coordinates"
  ) +
#geom_sf(data = clipped_total, aes(fill=clipped_total$flag)) +
colScale
#scale_size_continuous(range=c(1,12))
#theme(legend.position = "none") 
map

In [ ]:
clipped_total_flag <- clipped_total%>%
  group_by(flag) %>%
  summarise(totalFishingHours=sum(totalFishingHours))
clipped_total_flag

In [ ]:
#filter aggregated stats to bbox
dongwon_aggregated_bbox<-dongwon_aggregated[which(dongwon_aggregated$lat_bin > -9.84 & dongwon_aggregated$lat_bin < 4.7 & dongwon_aggregated$lon_bin > -176.85 & dongwon_aggregated$lon_bin < -151.41),]
aggregated_bbox<-aggregated[which(aggregated$lat_bin > -9.84 & aggregated$lat_bin < 4.7 & aggregated$lon_bin > -176.85 & aggregated$lon_bin < -151.41),]

aggregated_bbox_flag<-aggregated_bbox%>%
  group_by(flag) %>%
  summarise(totalFishingHours=sum(totalFishingHours))

dongwon_mmsi_BB<-dongwon_fishing[which(dongwon_fishing$lat_bin/10 > -9.84 & dongwon_fishing$lat_bin/10 < 4.7 & dongwon_fishing$lon_bin/10 > -176.85 & dongwon_fishing$lon_bin/10 < -151.41),]
dongwon_mmsi<-dongwon_mmsi_BB %>%
  group_by(mmsi) %>%
  summarise(totalFishingHours=sum(fishing_hours))